In [1]:
import pprint
sys.path.append('/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/')
pprint.pprint(sys.path)

['/Users/mkumada/Documents/class/2020/work/combo3/mywork',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles',
 '/Users/mkumada/.vscode/extensions/ms-python.python-2020.8.108011/pythonFiles/lib/python',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python36.zip',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6',
 '/Users/mkumada/.pyenv/versions/3.6.8/lib/python3.6/lib-dynload',
 '',
 '/Users/mkumada/.pyenv/versions/overlap_detect/lib/python3.6/site-packages',
 '/Users/mkumada/.pyenv/versions/overlap_detect/lib/python3.6/site-packages/IPython/extensions',
 '/Users/mkumada/.ipython',
 '/Users/mkumada/.pyenv/versions/3.6.8/envs/combo3/lib/python3.6/site-packages/']


In [411]:
import sys
import numpy as np
import _pickle as cPickle
import scipy
import combo
import os
import urllib
import time
from combo.variable import variable
import random
import csv
from decimal import *

In [383]:
# データ取得
def load_data(path):
    A = np.loadtxt(path,skiprows=1,delimiter=',')
    if A.ndim==1:
        A = np.expand_dims(A, axis=0)
    X = A[:, :-1]
    y  = -A[:, -1]
    return X, y, A

In [377]:
# 濃度変換
def concentration_conversion(X):
    if X.ndim==1:
        X = X/np.sum(X)
    else:
        X = X/np.expand_dims(np.sum(X ,axis=1), axis=1)
    return X

In [258]:
# delta
def cost_func(a, b, c):
    return np.sqrt((0.2-a)**2+(0.3-b)**2+(0.5-c)**2)

In [433]:
# N種類の材料をランダムに1ずつ添加する
N = 3
ITER_NUM = 5
TRAIN_PATH = './data/train2.csv'

for _ in range(ITER_NUM):
    select_arr = np.eye(N, dtype=float) # one hot matrix
    X_train, _, _ = load_data(TRAIN_PATH) # read data
    update_quantity = X_train[-1] + select_arr[random.randrange(N)] # N種類の材料から１つをランダムに選び、1足す.
    update_quantity_ratio = concentration_conversion(update_quantity) # 濃度に変換する.
    delta = cost_func(update_quantity_ratio[0], update_quantity_ratio[1], update_quantity_ratio[2]) # 目指す匂いの目的関数との誤差
    new_data= np.append(update_quantity, delta)

    print(new_data)

    with open(TRAIN_PATH, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(new_data)

[2.         3.         2.         0.26418917]
[2.         2.         3.         0.11248583]
[2.         3.         2.         0.26418917]
[3.         2.         2.         0.31363569]
[2.         3.         2.         0.26418917]


In [431]:
# N種類の材料からベイズ最適化で添加する
N = 3
ITER_NUM = 50
TRAIN_PATH = './data/train2.csv'
OUT_PATH = './data/out2.csv'
THRESHOLD_VAL =  0.15
SCALE_FACTOR =  0.1

for _ in range(ITER_NUM):

    X_train, t_train, X = load_data(OUT_PATH)

    if np.abs(t_train[-1]) > THRESHOLD_VAL:
        X_test = np.eye(N, dtype=float) # one hot matrix
    else:
        X_test = SCALE_FACTOR * np.eye(N, dtype=float) # one hot matrix

    X_train_ratio = concentration_conversion(X_train)

    # feature vector の各要素を正規化します。
    ave = np.mean(X_train_ratio, axis=0)
    std = np.std(X_train_ratio, axis=0)
    X_train_ratio = (X_train_ratio - ave) / (std + 1e-8)
    X_test_ratio = (X_test - ave) / (std + 1e-8)

    train_data = variable(X=X_train_ratio, t=t_train)
    test_data = variable(X=X_test_ratio)

    policy = combo.search.discrete.policy(test_X=test_data)
    # set the seed parameter
    policy.set_seed(0)
    action = policy.bayes_search_single(training=train_data, score='TS', num_rand_basis=5000)
    # score: 獲得関数(acquisition function) のタイプ。
    # TS (Thompson Sampling)
    # EI (Expected Improvement)
    # PI (Probability of Improvement)

    print(action)
    print(X_test[action])

    update_quantity = X_train[-1] + X_test[action]
    update_quantity_ratio = concentration_conversion(update_quantity) # 濃度に変換する.
    delta = cost_func(update_quantity_ratio[0], update_quantity_ratio[1], update_quantity_ratio[2]) # 目指す匂いの目的関数との誤差

    new_data = np.append(np.round(update_quantity, decimals=5), delta)

    with open(OUT_PATH, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(new_data)


.0311232454155
150-th epoch, marginal likelihood -78.11347556117555
200-th epoch, marginal likelihood -79.96195753459483
250-th epoch, marginal likelihood -81.64197872701935
300-th epoch, marginal likelihood -83.19739845008046
350-th epoch, marginal likelihood -84.65821486923515
400-th epoch, marginal likelihood -86.04535285490628
450-th epoch, marginal likelihood -87.37367852578757
500-th epoch, marginal likelihood -88.6539293527623
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.]
 scale  =  1.0
 scale2 =  1.0
 

Fitting ends
Calculating score for test data
2
[0.  0.  0.1]
Fitting starts
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood -73.79065597567639
50-th epoch, marginal likelihood -76.77316056137329
100-th epoch, marginal likelihood -79.27061433786862
150-th epoch, marginal likelihood -81.42573337642435
200-th epoch, marginal likelihood -83.34022041411713
250-th epoch, marginal likelihood -85.0813